In [1]:
from LiveAMP import *
FLAGS().run()
self = AMP(**kwargs)
# self.get_terms()
# self.get_raw_df()
# self.get_reg_df()
# self.get_X()
# self.get_Y()
# self.get_mlt()
# self.get_transformed()
# self.get_imputed()
# self.get_targets()
# self.get_predicted()
# self.get_optimal()
# self.get_details()
self.get_summary()
# self.get_params()
# self.push()
# self.get_anonymized()

summary, all; 

,ct,pct
pred_code,,
202108,5400,25
202208,5400,25
202308,5400,25
202408,5400,25


complete: 1 / 1 = 100.00%; elapsed = 3.23 min; remaining = 0.00 min @ 3.2307 min per iteration


In [3]:
S = self['summary']['all']
describe = [pctl(0),pctl(25),pctl(50),pctl(75),pctl(100)]
grp = ['crse_code','levl_desc','styp_desc','train_desc']
E = S.query(f"pred_code==202308").rename(columns={'error_pct':'2023 error_pct'}).groupby(self.mlt_grp)[['2023 error_pct']].agg(describe).droplevel('pred_code')
P = S.query(f"pred_code==202408").rename(columns={'predicted':'2024 predicted'}).groupby(self.mlt_grp)[['2024 predicted']].agg(describe).droplevel('pred_code')
A = S.query(f"pred_code==202308").rename(columns={'actual':'2023 actual'}).groupby(self.mlt_grp)[['2023 actual']].agg([pctl(50)]).droplevel('pred_code')
Q = E.join(A).join(P)
Q['increase'] = Q["2024 predicted"," 50%"] - Q["2023 actual"," 50%"]
Q['increase_percent'] = 100 * Q['increase'] / Q["2023 actual"," 50%"]
Q = Q.round(1).query("styp_code=='n'")
Q
write(self.root / 'amp_projections_2024_04_10.csv', Q)

2023 error_pct                           \
                                          0%    25%   50%   75%  100%   
crse_code levl_code styp_code                                           
_allcrse  ug        n                    0.4    1.0   6.8   8.4   9.1   
agec2317  ug        n                   -2.5    0.5   2.6   5.5   9.8   
ansc1119  ug        n                   -2.0   -0.7  -0.2   0.0   1.3   
ansc1319  ug        n                   -0.9   -0.6   0.7   0.9   2.3   
arts1301  ug        n                   -2.8    1.9  15.7  44.5  48.6   
biol1406  ug        n                   -0.8    0.7   1.6   9.7  11.3   
biol1407  ug        n                   -4.1   -4.1   0.0   3.7   6.9   
biol2401  ug        n                   -2.5   -0.9  -0.3   0.2   2.3   
biol2402  ug        n                  -13.0   -4.8   0.0   1.4   6.3   
busi1301  ug        n                   -1.8    0.5   3.9  16.7  18.9   
chem1111  ug        n                   -3.9   -1.6   0.0   2.4   4.9   
chem1302  ug        n                   -2.7    2.7   5.8   8.6   9.8   
chem1311  ug        n                   -5.6   -3.4  -1.2   1.2   3.6   
comm1311  ug        n                   -0.6    4.1   6.2   9.5  13.6   
comm1315  ug        n                   -2.4    0.0   4.7  11.1  13.3   
comm2302  ug        n                  -18.3  -14.0 -10.8  -1.7   3.3   
crij1301  ug        n                   -4.8   -1.2   1.3   4.7   9.0   
dram1310  ug        n                   -6.5   -2.1   2.1  13.9  23.4   
econ2301  ug        n                   -2.2   -2.2   0.0   3.1   8.2   
engl1301  ug        n                   -2.8    0.6   4.6   6.7   9.1   
engl1302  ug        n                   -2.5    1.2   6.1   9.9  14.1   
geol1403  ug        n                   -1.1    1.1  15.8  21.4  24.1   
geol1404  ug        n                    0.0    0.0  12.2  27.0  27.0   
geol1407  ug        n                   -2.5    2.5  19.7  62.5  70.8   
geol1408  ug        n                   -1.2   -1.2   0.0   9.2  11.9   
govt2305  ug        n                   -3.3    2.6  14.7  26.5  33.9   
govt2306  ug        n                   -2.3    1.3   5.2  10.4  17.2   
hist1301  ug        n                   -2.2   -0.6   5.5  13.9  15.8   
hist1302  ug        n                   -3.9   -2.2   0.0   1.7   3.7   
huma1315  ug        n                  -11.7   -7.1  -5.8   0.0   0.0   
math1314  ug        n                   -2.7   -0.5   1.1   6.3   8.5   
math1316  ug        n                  -17.8  -10.9 -10.0  -3.8   3.8   
math1324  ug        n                   -0.9    1.4  15.5  22.7  25.6   
math1342  ug        n                  -12.2   -7.7  -4.7  -2.6   3.1   
math2412  ug        n                   -1.3    1.6   5.9   8.4  13.2   
math2413  ug        n                   -1.5    1.5  10.9  25.3  33.0   
musi1310  ug        n                   -9.2   -5.1  -0.3   9.0  27.9   
musi1311  ug        n                   -7.4   -5.2   0.0  15.5  15.5   
phil1301  ug        n                   -4.4   -0.0   2.0  20.3  29.2   
phil2303  ug        n                   -9.1   -9.1   0.0  21.2  21.2   
psyc2301  ug        n                   -1.5    0.3   2.9   6.4   8.6   
soci1301  ug        n                   -3.4    0.0  23.8  26.3  32.0   
univ0200  ug        n                   -6.2   -1.2   1.3   3.8   9.1   
univ0204  ug        n                   -5.7   -0.6   3.0   5.8  14.4   
univ0301  ug        n                   -5.0   -1.1   0.7   2.9   5.5   
univ0314  ug        n                  -18.1  -13.3   0.5   2.1   3.2   
univ0342  ug        n                 -100.0 -100.0   0.0   5.9  12.8   

                              2023 actual 2024 predicted                  \
                                      50%             0%     25%     50%   
crse_code levl_code styp_code                                              
_allcrse  ug        n              2700.0         3001.9  3049.2  3202.4   
agec2317  ug        n               212.0          248.0   264.6   270.7   

In [10]:
M = self.mlt['all'].reset_index()
M

,crse_code,levl_code,styp_code,pred_code,train_code,mlt_code,mlt_actual,mlt_predicted
0,_allcrse,ug,n,202108,202108,202108,1.213477,1.213477
1,_allcrse,ug,n,202108,202108,202208,1.213477,1.191821
2,_allcrse,ug,n,202108,202108,202308,1.213477,1.126408
3,_allcrse,ug,n,202108,202208,202108,1.191821,1.213477
4,_allcrse,ug,n,202108,202208,202208,1.191821,1.191821
...,...,...,...,...,...,...,...,...
6683,univ0342,ug,r,202408,202308,202308,<NA>,<NA>
6684,univ0342,ug,t,202408,202208,202208,1.5,1.5
6685,univ0342,ug,t,202408,202208,202308,1.5,<NA>
6686,univ0342,ug,t,202408,202308,202208,<NA>,1.5


In [14]:
Y = self.get(path | {'nm':'details'})

In [20]:
# Y.reset_index(set(Y.index.names).difference(self.summary_grp), drop=True)
Y.reset_index([k for k in Y.index.names if k not in self.summary_grp], drop=True).join(self.mlt['all'])

predicted  \
pred_code levl_code styp_code imp crse_code train_code mlt_code              
202408    ug        n         4   _allcrse  202308     202108        False   
                                                       202208        False   
                                                       202308        False   
202208    ug        n         5   _allcrse  202308     202108        False   
                                                       202208        False   
...                                                                    ...   
202308    ug        n         4   univ0342  202308     202208        False   
                                                       202308        False   
202208    ug        n         3   univ0342  202308     202108        False   
                                                       202208        False   
                                                       202308        False   

                                                                 actual  \
pred_code levl_code styp_code imp crse_code train_code mlt_code           
202408    ug        n         4   _allcrse  202308     202108     False   
                                                       202208     False   
                                                       202308     False   
202208    ug        n         5   _allcrse  202308     202108     False   
                                                       202208     False   
...                                                                 ...   
202308    ug        n         4   univ0342  202308     202208     False   
                                                       202308     False   
202208    ug        n         3   univ0342  202308     202108     False   
                                                       202208     False   
                                                       202308     False   

                                                                 mlt_actual  \
pred_code levl_code styp_code imp crse_code train_code mlt_code               
202408    ug        n         4   _allcrse  202308     202108      1.126408   
                                                       202208      1.126408   
                                                       202308      1.126408   
202208    ug        n         5   _allcrse  202308     202108      1.126408   
                                                       202208      1.126408   
...                                                                     ...   
202308    ug        n         4   univ0342  202308     202208       1.06383   
                                                       202308       1.06383   
202208    ug        n         3   univ0342  202308     202108       1.06383   
                                                       202208       1.06383   
                                                       202308       1.06383   

                                                                 mlt_predicted  
pred_code levl_code styp_code imp crse_code train_code mlt_code                 
202408    ug        n         4   _allcrse  202308     202108         1.213477  
                                                       202208         1.191821  
                                                       202308         1.126408  
202208    ug        n         5   _allcrse  202308     202108         1.213477  
                                                       202208         1.191821  
...                                                                        ...  
202308    ug        n         4   univ0342  202308     202208         1.151515  
                                                       202308          1.06383  
202208    ug        n         3   univ0342  202308     202108             <NA>  
                                                       202208         1.151515  
                                                       202308          1.0

In [11]:
path = {'nm':'summary', 'styp_code':'all'}
Y = self.get(path | {'nm':'details'})

In [13]:
self.mlt['all']

mlt_actual  \
crse_code levl_code styp_code pred_code train_code mlt_code               
_allcrse  ug        n         202108    202108     202108      1.213477   
                                                   202208      1.213477   
                                                   202308      1.213477   
                                        202208     202108      1.191821   
                                                   202208      1.191821   
...                                                                 ...   
univ0342  ug        r         202408    202308     202308          <NA>   
                    t         202408    202208     202208           1.5   
                                                   202308           1.5   
                                        202308     202208          <NA>   
                                                   202308          <NA>   

                                                             mlt_predicted  
crse_code levl_code styp_code pred_code train_code mlt_code                 
_allcrse  ug        n         202108    202108     202108         1.213477  
                                                   202208         1.191821  
                                                   202308         1.126408  
                                        202208     202108         1.213477  
                                                   202208         1.191821  
...                                                                    ...  
univ0342  ug        r         202408    202308     202308             <NA>  
                    t         202408    202208     202208              1.5  
                                                   202308             <NA>  
                                        202308     202208              1.5  
                                                   202308             <NA>  

[6688 rows x 2 columns]

In [14]:
path = {'nm':'summary', 'styp_code':'all'}
Y = self.get(path | {'nm':'details'})
Y = Y.reset_index([k for k in Y.index.names if k not in self.summary_grp], drop=True).join(self.mlt['all']).reset_index().copy()
# Y = self.get(path | {'nm':'details'}).reset_index(self.summary_grp).reset_index(drop=True).copy()#.merge(M)

In [16]:
Y

,pred_code,levl_code,styp_code,imp,crse_code,train_code,mlt_code,predicted,actual,mlt_actual,mlt_predicted
0,202408,ug,n,4,_allcrse,202308,202108,False,False,1.126408,1.213477
1,202408,ug,n,4,_allcrse,202308,202208,False,False,1.126408,1.191821
2,202408,ug,n,4,_allcrse,202308,202308,False,False,1.126408,1.126408
3,202208,ug,n,5,_allcrse,202308,202108,False,False,1.126408,1.213477
4,202208,ug,n,5,_allcrse,202308,202208,False,False,1.126408,1.191821
...,...,...,...,...,...,...,...,...,...,...,...
39883045,202308,ug,n,4,univ0342,202308,202208,False,False,1.06383,1.151515
39883046,202308,ug,n,4,univ0342,202308,202308,False,False,1.06383,1.06383
39883047,202208,ug,n,3,univ0342,202308,202108,False,False,1.06383,<NA>
39883048,202208,ug,n,3,univ0342,202308,202208,False,False,1.06383,1.151515


In [17]:
for k in ['predicted','actual']:
    Y['mlt_'+k] *= Y[k]
Y

,pred_code,levl_code,styp_code,imp,crse_code,train_code,mlt_code,predicted,actual,mlt_actual,mlt_predicted
0,202408,ug,n,4,_allcrse,202308,202108,False,False,0.0,0.0
1,202408,ug,n,4,_allcrse,202308,202208,False,False,0.0,0.0
2,202408,ug,n,4,_allcrse,202308,202308,False,False,0.0,0.0
3,202208,ug,n,5,_allcrse,202308,202108,False,False,0.0,0.0
4,202208,ug,n,5,_allcrse,202308,202208,False,False,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
39883045,202308,ug,n,4,univ0342,202308,202208,False,False,0.0,0.0
39883046,202308,ug,n,4,univ0342,202308,202308,False,False,0.0,0.0
39883047,202208,ug,n,3,univ0342,202308,202108,False,False,0.0,<NA>
39883048,202208,ug,n,3,univ0342,202308,202208,False,False,0.0,0.0


In [5]:
P = [Y]
# # P = [q for p in P for q in [p, p.assign(styp_code ='all')]]
# # P = [q for p in P for q in [p, p.assign(train_code='all')]]
P[0]
with warnings.catch_warnings(action='ignore'):
    S = pd.concat([self.summarize(p) for p in P])

In [6]:
S.loc[S.eval("train_code=='all'"), ['predicted','actual','error']] /= Y['train_code'].nunique()
S.loc[S.eval(f"actual==0 or pred_code=={self.pred_code}"), 'actual':] = pd.NA
S = S.reset_index()
S['levl_desc'] = S['levl_code'].map({'ug':'undergraduate', 'g':'graduate'})
S['styp_desc'] = S['styp_code'].map({'n':'new first time', 't':'transfer', 'r':'returning', 'all':'all incoming'})
for k in ['pred','train','mlt']:
    S[k+'_desc'] = 'Fall ' + S[k+'_code'].astype('string').str[:4]
S.vc('pred_code').disp(100)
S = S.set_index(['crse_code','levl_code','levl_desc','styp_code','styp_desc','pred_code','pred_desc','train_code','train_desc','mlt_code','mlt_desc','imp'])
S

,ct,pct
pred_code,,
202108,1350,25
202208,1350,25
202308,1350,25
202408,1350,25


predicted  \
crse_code levl_code levl_desc     styp_code styp_desc      pred_code pred_desc train_code train_desc mlt_code mlt_desc  imp              
_allcrse  ug        undergraduate n         new first time 202108    Fall 2021 202308     Fall 2023  202108   Fall 2021 0       1542.0   
                                                                                                                        1       1535.0   
                                                                                                                        2       1531.0   
                                                                                                                        3       1544.0   
                                                                                                                        4       1557.0   
...                                                                                                                                ...   
univ0342  ug        undergraduate n         new first time 202408    Fall 2024 202308     Fall 2023  202308   Fall 2023 5         61.0   
                                                                                                                        6         64.0   
                                                                                                                        7         63.0   
                                                                                                                        8         65.0   
                                                                                                                        9         60.0   

                                                                                                                             actual  \
crse_code levl_code levl_desc     styp_code styp_desc      pred_code pred_desc train_code train_desc mlt_code mlt_desc  imp           
_allcrse  ug        undergraduate n         new first time 202108    Fall 2021 202308     Fall 2023  202108   Fall 2021 0    1766.0   
                                                                                                                        1    1766.0   
                                                                                                                        2    1766.0   
                                                                                                                        3    1766.0   
                                                                                                                        4    1766.0   
...                                                                                                                             ...   
univ0342  ug        undergraduate n         new first time 202408    Fall 2024 202308     Fall 2023  202308   Fall 2023 5       NaN   
                                                                                                                        6       NaN   
                                                                                                                        7       NaN   
                                                                                                                        8       NaN   
                                                                                                                        9       NaN   

                                                                                                                             error  \
crse_code levl_code levl_desc     styp_code styp_desc      pred_code pred_desc train_code train_desc mlt_code mlt_desc  imp          
_allcrse  ug        undergraduate n         new first time 202108    Fall 2021 202308     Fall 2023  202108   Fall 2021 0   -224.0   
                                                                                                                        1   -231.0   
                                      

In [9]:
S.query("crse_code=='_allcrse'")

predicted  \
crse_code levl_code levl_desc     styp_code styp_desc      pred_code pred_desc train_code train_desc mlt_code mlt_desc  imp              
_allcrse  ug        undergraduate n         new first time 202108    Fall 2021 202308     Fall 2023  202108   Fall 2021 0       1542.0   
                                                                                                                        1       1535.0   
                                                                                                                        2       1531.0   
                                                                                                                        3       1544.0   
                                                                                                                        4       1557.0   
...                                                                                                                                ...   
                                                           202408    Fall 2024 202308     Fall 2023  202308   Fall 2023 5       2696.0   
                                                                                                                        6       2683.0   
                                                                                                                        7       2707.0   
                                                                                                                        8       2665.0   
                                                                                                                        9       2689.0   

                                                                                                                             actual  \
crse_code levl_code levl_desc     styp_code styp_desc      pred_code pred_desc train_code train_desc mlt_code mlt_desc  imp           
_allcrse  ug        undergraduate n         new first time 202108    Fall 2021 202308     Fall 2023  202108   Fall 2021 0    1766.0   
                                                                                                                        1    1766.0   
                                                                                                                        2    1766.0   
                                                                                                                        3    1766.0   
                                                                                                                        4    1766.0   
...                                                                                                                             ...   
                                                           202408    Fall 2024 202308     Fall 2023  202308   Fall 2023 5       NaN   
                                                                                                                        6       NaN   
                                                                                                                        7       NaN   
                                                                                                                        8       NaN   
                                                                                                                        9       NaN   

                                                                                                                             error  \
crse_code levl_code levl_desc     styp_code styp_desc      pred_code pred_desc train_code train_desc mlt_code mlt_desc  imp          
_allcrse  ug        undergraduate n         new first time 202108    Fall 2021 202308     Fall 2023  202108   Fall 2021 0   -224.0   
                                                                                                                        1   -231.0   
                                      

2023 error_pct                      2023 actual  \
                                          0%  25%  50%  75%  100%         50%   
crse_code levl_code styp_code                                                   
_allcrse  ug        n                    0.4  0.6  0.9  1.0   1.3      2397.0   
agec2317  ug        n                   -2.5 -1.5 -0.5  0.5   2.5       200.0   
ansc1119  ug        n                   -2.0 -0.7 -0.2  0.0   1.3       306.0   
ansc1319  ug        n                   -0.9 -0.4  0.7  0.9   2.3       344.0   
arts1301  ug        n                   -2.8  0.0  0.9  1.9   2.8       106.0   
biol1406  ug        n                   -0.8 -0.5  0.0  0.7   1.0       614.0   
biol1407  ug        n                    0.0  0.0  0.0  0.0   3.0        33.0   
biol2401  ug        n                   -1.9 -0.9 -0.2  0.2   2.1       432.0   
biol2402  ug        n                   -4.8  0.0  0.0  4.8   4.8        21.0   
busi1301  ug        n                   -1.8 -0.9 -0.5  0.5   0.9       220.0   
chem1111  ug        n                    0.0  1.2  2.4  2.4   4.9        82.0   
chem1302  ug        n                   -2.7  0.0  0.0  2.7   2.7        37.0   
chem1311  ug        n                   -1.2 -1.2  1.2  2.4   3.6        84.0   
comm1311  ug        n                   -0.6  1.9  2.8  4.4   6.4       362.0   
comm1315  ug        n                   -2.4 -1.2 -0.6  0.0   0.8       254.0   
comm2302  ug        n                   -5.0 -1.7 -0.8  1.7   3.3        60.0   
crij1301  ug        n                   -4.8 -3.6 -1.2  0.0   2.4        83.0   
dram1310  ug        n                   -2.1  0.0  1.0  4.2   8.3        48.0   
econ2301  ug        n                    0.0  0.0  0.0  0.0   5.0        20.0   
engl1301  ug        n                   -2.8 -1.4 -0.8  0.6   0.9      1172.0   
engl1302  ug        n                   -2.5 -1.2  0.6  1.2   2.5        81.0   
geol1403  ug        n                   -1.1  0.0  0.0  1.1   2.2        89.0   
geol1404  ug        n                    0.0  0.0  0.0  0.0   0.0        20.0   
geol1407  ug        n                   -2.5 -2.5  1.2  2.5   2.5        40.0   
geol1408  ug        n                    0.0  0.0  0.0  0.0   2.5        40.0   
govt2305  ug        n                   -3.3  0.0  0.7  2.6   5.9       153.0   
govt2306  ug        n                   -2.3 -1.3  0.2  1.3   4.8       310.0   
hist1301  ug        n                   -2.2 -1.7 -1.1 -0.6   0.0       360.0   
hist1302  ug        n                   -1.7 -1.7  0.0  0.0   1.7        59.0   
huma1315  ug        n                   -5.0  0.0  0.0  0.0   0.0        20.0   
math1314  ug        n                   -2.7 -2.2 -1.5 -0.5  -0.2       639.0   
math1316  ug        n                   -7.7 -3.8  0.0  0.0   3.8        26.0   
math1324  ug        n                   -0.9 -0.5 -0.5  1.4   1.9       214.0   
math1342  ug        n                   -6.8 -3.1 -2.1  1.6   3.1       191.0   
math2412  ug        n                   -1.3 -1.3 -0.6  1.6   3.8       316.0   
math2413  ug        n                   -1.5 -1.5  0.0  1.5   4.5        66.0   
musi1310  ug        n                   -5.1 -3.8 -3.2  0.0  12.8        78.0   
musi1311  ug        n                   -2.3  0.0  0.0  0.0   0.0        44.0   
phil1301  ug        n                   -4.4 -1.5  0.0  1.5   5.9        68.0   
phil2303  ug        n                    0.0  0.0  0.0  0.0   0.0        20.0   
psyc2301  ug        n                   -1.5 -0.9 -0.1  0.3   1.2       339.0   
soci1301  ug        n                   -3.4 -1.7  0.0  0.0   3.4        59.0   
univ0200  ug        n                   -3.4 -0.9  0.4  4.3   6.0       117.0   
univ0204  ug        n                   -5.7 -2.0 -1.2  2.8   8.5       247.0   
univ0301  ug        n                   -5.0 -2.2 -0.5  2.5   2.9       279.0   
univ0314  ug        n                   -3.4  0.0  1.1  2.2   2.2        89.0   
univ0342  ug        n                   -4.3 -2.1  0.0  2.1  

In [ ]:
A

In [ ]:
self['summary']['all']

In [ ]:
M = self.mlt['all']
M.query("crse_code=='math1314' & styp_code=='n'")

In [ ]:
A = self['details']['all'].query("crse_code=='math1314'")
A.vc('pred_code')

In [ ]:
Y = A.join(M).reset_index(self.summary_grp).reset_index(drop=True)#.set_index(self.summary_grp)

In [ ]:
for k in ['predicted','actual']:
    Y['mlt_'+k] = Y['mlt_'+k] * Y[k]
Y.disp(1)

In [ ]:
S = self.summarize(Y)
S.disp(200)

In [ ]:
S.loc[S.eval("train_code=='all'"), ['predicted','actual','error']] /= Y['train_code'].nunique()
S.loc[S.eval(f"actual==0 or pred_code=={self.pred_code}"), 'actual':] = pd.NA
S = S.reset_index()
S['levl_desc'] = S['levl_code'].map({'ug':'undergraduate', 'g':'graduate'})
S['styp_desc'] = S['styp_code'].map({'n':'new first time', 't':'transfer', 'r':'returning', 'all':'all incoming'})
for k in ['pred','train','mlt']:
    S[k+'_desc'] = 'Fall ' + S[k+'_code'].astype('string').str[:4]
S.vc('pred_code').disp(100)
A = S.set_index(['crse_code','levl_code','levl_desc','styp_code','styp_desc','pred_code','pred_desc','train_code','train_desc','mlt_code','mlt_desc','imp'])


In [ ]:
B.missing()
mask = B.isnull().any(axis=1)
B[mask].reset_index()[self.mlt_grp].dtypes.disp(10)

In [ ]:
M.query("crse_code=='agec2317'").reset_index().dtypes

In [ ]:
B.missing()

In [ ]:
C = B.copy()
for k in ['predicted','actual']:
    C['mlt_'+k] = C['mlt_'+k] * C[k]
C.vc('pred_code').disp(100)

In [ ]:
Y = C.copy()
repl = {k+'_mlt':k for k in ['predicted','actual']} | {'mlt_code':'train_code'}
for k, v in repl.items():
    if k not in Y:
        Y[k] = Y[v]
Y.vc('pred_code').disp(100)

In [ ]:
Z = Y.groupby(self.summary_grp, dropna=False)[['predicted_mlt']].sum()
Z.vc('pred_code').disp(100)

In [ ]:
Z.query("crse_code=='agec2317'").disp(500)

In [ ]:
Z.groupby(self.mlt_grp, dropna=False).size().to_frame().query("crse_code=='agec2317'")
# M = self.mlt['all']
# # .reset_index().missing()
# mask = M.isnull().any(axis=1)
# M[mask].sort_index()

In [ ]:
Z.query('pred_code<202408').groupby(['crse_code','pred_code','mlt_code'], dropna=False).size().disp(200)
# .groupby('pred_code', dropna=False).size().disp(200)

# 


In [ ]:
A